In [25]:
import pandas as pd
import sqlite3

In [26]:
con = sqlite3.connect("../data/checking-logs.sqlite")

In [27]:
pageviews = pd.read_sql("PRAGMA table_info(test);",con=con)
pageviews

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


In [5]:
pd.read_sql("SELECT * FROM test LIMIT 10",con)

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


Find the minimum value of the delta between the first commit and the deadline of the corresponding lab for all users using only one query.
Do this by joining the table with the deadlines table.
The difference should be displayed in hours.
Do not take lab project1 into account; it has longer deadlines and will be an outlier.
The value should be stored in the dataframe df_min with the corresponding uid.

In [6]:
pd.read_sql("SELECT * FROM deadlines LIMIT 10",con)

,index,labs,deadlines
0,0,laba04,1587945599
1,1,laba04s,1587945599
2,2,laba05,1588550399
3,4,laba06,1590364799
4,5,laba06s,1590364799
5,3,project1,1589673599


In [20]:
df_min = pd.read_sql("""SELECT t.uid,MIN((julianday(d.deadlines, 'unixepoch') - julianday(t.first_commit_ts)) * 24) AS min_delta_hours FROM test t
JOIN deadlines d ON t.labname = d.labs  
WHERE d.labs != 'project1'
GROUP BY t.uid """,con)
df_min

,uid,min_delta_hours
0,user_1,6.796432
1,user_10,39.367888
2,user_14,84.448466
3,user_17,34.643043
4,user_18,3.933907
5,user_19,32.729282
6,user_21,33.905274
7,user_25,2.867236
8,user_28,8.103915
9,user_3,60.511392


In [42]:
df_max = pd.read_sql("""SELECT t.uid,MAX((julianday(d.deadlines, 'unixepoch') - julianday(t.first_commit_ts)) * 24) AS max_delta_hours FROM test t
JOIN deadlines d ON t.labname = d.labs  
WHERE d.labs != 'project1'
GROUP BY t.uid """,con)
df_max

,uid,max_delta_hours
0,user_1,175.556592
1,user_10,132.341698
2,user_14,200.766302
3,user_17,81.591403
4,user_18,10.973375
5,user_19,148.916028
6,user_21,126.199587
7,user_25,150.869725
8,user_28,174.852984
9,user_3,182.055144


In [41]:
df_avg = pd.read_sql("""SELECT AVG((julianday(d.deadlines, 'unixepoch') - julianday(t.first_commit_ts)) * 24) AS avg_delta_hours FROM test t
JOIN deadlines d ON t.labname = d.labs  
WHERE d.labs != 'project1'
GROUP BY t.uid """,con)
df_avg

,avg_delta_hours
0,65.119644
1,75.242310
2,159.568696
3,62.207513
4,6.367907
5,99.440298
6,96.111041
7,93.474751
8,86.793652
9,105.738041


In [44]:
views_diff = pd.read_sql(sql="""SELECT t.uid,AVG((julianday(d.deadlines, 'unixepoch') - julianday(t.first_commit_ts)) * 24) AS avg_delta_hours,
                        FROM test t 
                         LEFT JOIN deadlines d ON d.labs = t.labname
                         LEFT JOIN pageviews p ON p.uid = t.uid
                         WHERE d.labs != 'project1'
                         GROUP BY t.uid
                         """,con=con)
views_diff

DatabaseError: Execution failed on sql 'SELECT t.uid,AVG((julianday(d.deadlines, 'unixepoch') - julianday(t.first_commit_ts)) * 24) AS avg_delta_hours,
                          COALESCE(SUM(p.pageviews), 0) AS pageviews FROM test t 
                         LEFT JOIN deadlines d ON d.labs = t.labname
                         LEFT JOIN pageviews p ON p.uid = t.uid
                         WHERE d.labs != 'project1'
                         GROUP BY t.uid
                         ': no such column: p.pageviews

In [23]:
con.close()